In [ ]:
TIME_TO_MEASURE = 9

class State:
    """
    The attribute `small` refers to the time that is going to pass until all the sand has fallen in the small
    hourglass. For example `small == 0` means that we just swapped the hourglass and it will be done in 4 minutes.
    `small == 0` means that the hourglass is not measuring. Simirally we define `big`: the bigger hourglass. The
    attribute `time` signifies the time already measured since the beggining of the simulation.
    """
    SMALL_CAPACITY = 4
    BIG_CAPACITY = 7
    
    def __init__(self, small, big, time):
        self.small = small
        self.big = big
        self.time = time
    
    def __eq__(self, other):
        """Enable equality checks by overriding default behavior."""
        if isinstance(other, State):
            return self.__dict__ == other.__dict__
        
        return False
    
    def __hash__(self):
        return hash((self.small, self.big, self.time))
    
    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    def success(self):
        return self.time == TIME_TO_MEASURE
    
    def neighbors(self, available_moves):
        return [move(self) for move in available_moves]
    
    def swap_big(self):
        return State(self.small, self.BIG_CAPACITY - self.big, self.time)
    
    def swap_small(self):
        return State(self.SMALL_CAPACITY - self.small, self.big, self.time)
        
    def __str__(self):
        """Facilitate printing for debugging purposes"""
        return("Small: {}, Big: {}, Time Measured: {}".format(self.small, self.big, self.time))
        
initial_state = State(small=0, big=0, time=0)
print(initial_state)    

In [ ]:
def do_nothing(state, debug=True):
    """
    If we don't do anything, we just wait until one of the two hourglasses finish (obviously the one with 
    lower time left)
    """
    if state.small == 0 and state.big == 0:
        # Nothing happened.
        return state
    
    if state.big == 0:
        # Finish the small one.
        return State(small=0, big=0, time=state.time + state.small)
    
    if state.small == 0:
        # Finish the big one.
        return State(small=0, big=0, time=state.time + state.big)
    
    if state.big < state.small:
        # The big one will finish first.
        return State(small=state.small - state.big, big=0, time=state.time + state.big)
    
    # The small one will finish first.
    return State(small=0, big = state.big - state.small, time=state.time + state.small)

def test_do_nothing():
    """Unit test the most  boring move ever."""
    state = State(small=4, big=3, time=5)
    assert do_nothing(state) == State(small=1, big=0, time=8)
    
    state = State(small=0, big=6, time=2)
    assert do_nothing(state) == State(small=0, big=0, time=8)

test_do_nothing()    

In [ ]:
def swap_big(state):
    return do_nothing(state.swap_big(), debug=False)

def swap_small(state):
    return do_nothing(state.swap_small(), debug=False)

def swap_both(state):
    return do_nothing(state.swap_small().swap_big(), debug=False)

def test_swap():
    state = State(0, 0, 0)
    assert swap_big(state) == State(0, 0, 7)
    assert swap_small(state) == State(0, 0, 4)
    assert swap_both(state) == State(0, 3, 4)
    
    state = State(4, 2, 1)
    assert swap_big(state) == State(0, 1, 5)
    assert swap_small(state) == State(0, 0, 3)
    assert swap_both(state) == State(0, 0, 6)
    
test_swap()

In [ ]:
available_moves = [do_nothing, swap_big, swap_small, swap_both]

visited = []
stack = [initial_state]
parent = {'initial_state': None}

while stack:
    state = stack.pop(0)
    if state.success():
        print("Gotcha!")
        break
    for n in state.neighbors(available_moves):
        if n not in visited:
            stack.append(n)
            parent[n] = state
            visited.append(n)
    
    

In [ ]:
def find_move(before, after):
    for move in available_moves:
        if move(before) == after:
            return move

def print_move(move):
    if move == swap_big:
        print("Swap big")
    elif move == swap_small:
        print("Swap small")
    elif move == do_nothing:
        print("Do nothing")
    elif move == swap_both:
        print("Swap both")
        
path = []
previous = state
while previous != State(0, 0 ,0):
    #print(previous)
    path.append(previous)
    previous = parent[previous]
path.append(initial_state)


# Let's print out the path (list of moves)
path = list(reversed(path))
    
for before, after in zip(path, path[1:]):
    print_move(find_move(before, after))
    print(after)
print("TADA!")